## Blockchain vs SQL/SQLite Demo Notebook


### Introduction

#### Blockchain vs SQL/SQLite Demo

This notebook demonstrates:
- A minimal blockchain storing student records
- SQLite database storing the same student records
- Contrast between blockchain immutability and SQL database mutability


In [ ]:
# imports
import hashlib
import json
import time
import sqlite3

### Part 1: Blockchain Implementation

#### Part 1a: Instantiate Blockchain Class and Functions

In [ ]:

class Block():
    def __init__(self, index, data, previous_hash):
        self.index = index
        self.timestamp = time.time()
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.compute_hash()


    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()


class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_genesis_block()


def create_genesis_block(self):
    genesis = Block(0, {"message": "Genesis Block"}, "0")
    self.chain.append(genesis)


def add_block(self, data):
    prev_hash = self.chain[-1].hash
    new_block = Block(len(self.chain), data, prev_hash)
    self.chain.append(new_block)
    return new_block


def is_chain_valid(self):
    for i in range(1, len(self.chain)):
        current = self.chain[i]
        prev = self.chain[i-1]
        if current.hash != current.compute_hash():
            return False
        elif current.previous_hash != prev.hash:
            return False
        else:
            return True

#### Part 1b: Blockchain Demo

In [ ]:
bc = Blockchain()
bc.add_block({"student_id": 1, "name": "Alice", "grade": "A"})
bc.add_block({"student_id": 2, "name": "Bob", "grade": "B"})
bc.add_block({"student_id": 3, "name": "Charlie", "grade": "A"})


for block in bc.chain:
    print(f"Block {block.index}")
    print(f"Data: {block.data}")
    print(f"Hash: {block.hash[:10]}... | Prev: {block.previous_hash[:10]}...")
    print('\n', "Blockchain valid? ", bc.is_chain_valid())

#### Part 1c: Blockchain Tampering Validation

In [ ]:
print("\n--- Tampering with Block 1 ---")
bc.chain[1].data["name"] = "Eve"
print("Blockchain valid after tampering?", bc.is_chain_valid())

### Part 2: Standard Database with SQL Implementation

#### Part 2a: Instantiate Table for DB

In [ ]:
conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("CREATE TABLE students (id INTEGER PRIMARY KEY, name TEXT, grade TEXT)")

#### Part 2b: Insert Records into DB

In [ ]:
c.execute("INSERT INTO students VALUES (1, 'Alice', 'A')")
c.execute("INSERT INTO students VALUES (2, 'Bob', 'B')")
c.execute("INSERT INTO students VALUES (3, 'Charlie', 'A')")
conn.commit()

#### Part 2c: Basic Query

In [ ]:
# Query grade A students
c.execute("SELECT * FROM students WHERE grade='A'")
print("Students with grade A:", c.fetchall())

#### Part 2d: Update and Delete from DB

In [ ]:
c.execute("UPDATE students SET name='Eve' WHERE id=2")
c.execute("DELETE FROM students WHERE id=3")
conn.commit()

#### Part 2e: Query DB after Deletion

In [ ]:
# Query all records after updates
c.execute("SELECT * FROM students")
print("All students after updates:", c.fetchall())

### Part 3: Comparing BC and DB


| Feature | Blockchain | SQL/SQLite |
|---------|------------|------------|
| Data Mutability | Immutable once added | INSERT, UPDATE, DELETE allowed |
| Integrity | Cryptographic hashes | Constraints, triggers, ACID |
| Transaction Ordering | Sequential chain | DBMS log / isolation levels |
| Querying | Full scan or custom index | Optimized SQL queries |
| Replication | Consensus across nodes | Master-slave / clustering |


### Summary and Key Takeaways

- Blockchain ensures immutability and tamper-evidence.
- SQL DBs are flexible, efficient, and support ACID properties.
- Hybrid approaches exist, e.g., blockchain as audit logs for SQL databases.